In [1]:
# EXECUTE FIRST

# computational imports
from pyomo.environ import *
# for reading files from urls
import urllib.request
# display imports
from IPython.display import display, IFrame
from IPython.core.display import HTML

# for playing videos, customize height and width if desired
def play_video(vid_name, w = 640, h = 360):
    vid_path = "https://media.uwex.edu/content/ds/ds775_r19/"
    return IFrame( vid_path + vid_name + "/index.html", width = w, height = h )

# import notebook styling for tables and width etc.
response = urllib.request.urlopen('https://raw.githubusercontent.com/DataScienceUWL/DS775v2/master/ds755.css')
HTML(response.read().decode("utf-8"));

<font size=18>Lesson 06: Integer Programming</font>

# Key Integer Programming Concepts

* Decision variables constrained to integer values
    * Can produce 5 or 6 cars, but not 5.72 cars
* For pure integer programming (IP) problems, solutions can be obtained simply by changing the domain for the LP from **NonNegativeReals** to **PositiveIntegers** in the Pyomo coding (as seen in textbook problem 3.4-10 as a Self-Assessment back in Lesson 01)
* Binary Integer Programming (BIP) 
    * contain binary (boolean) variables  
    * *i.e.* 0 for no, 1 for yes
* Mixed Integer Programming (MIP)
    * some variables are constrained to be integers while other are not
* Computationally, integer programming can be much more difficult than linear programming (this <a href = https://www.quora.com/What-is-the-difference-between-integer-programming-and-linear-programming > post </a> can help you visualize why this is so)
* Binary variables are considerably easier to deal with than general integer variables, so they generally can be used to solve substantially larger problems

# Instructions for this Lesson

The material in this lesson is organized a bit differently than the other lessons.  There are 10 slides this week that have been organized into a "Storybook".  You'll be familiar with this format from previous courses.  
1. Study through slide 3 and then complete the first self-assessment problem at the bottom of this notebook.
2. Study the rest of the slides and work through the material in the section "Making Choices Using Continuous Decision Variables".
3. Complete the other self-assessements at the bottom of this notebook.
4. Complete and submit your homework.

# Storybook Presentation

In [2]:
# execute this cell to display storybook
play_video("ds775_lesson6",w=900,h=600)

# Making Choices with Continuous Decision Variables

This is the Good Products example (textbook page 489), similar to the Wyndor problem, in which we have to choose which products to produce and which factories to use.
The problem description is reproduced here for convenience.

## Problem Description

The Research and Development Division of the GOOD PRODUCTS COMPANY has developed three possible new products. However, to avoid undue diversification of the company’s product line, management has imposed the following restriction:

- **Restriction 1:** From the three possible new products, at most two should be chosen to be produced.

Each of these products can be produced in either of two plants. For administrative reasons, management has imposed a second restriction in this regard.

- **Restriction 2:** Just one of the two plants should be chosen to be the sole producer of the new products.

The production cost per unit of each product would be essentially the same in the two plants. However, because of differences in their production facilities, the number of hours of production time needed per unit of each product might differ between the two plants. These data are given in the table below, along with other relevant information, including marketing estimates of the number of units of each product that could be sold per week if it is produced. The objective is to choose the products, the plant, and the production rates of the chosen products so as to maximize total profit.

<img src="images/good_products_table.png" width="600">

## Mathematical Formulation

Maximize $Z = 5x_1 + 7x_2 + 3x_3$

Subject to:

$
\begin{array}{l}
x_1 \leq 7 \\
x_2 \leq 5 \\
x_3 \leq 9 \\
 y_1 + y_2 + y_3 = 2 \\
3x_1 + 4x_2 + 2x_3 \leq 30 + My_4 \\
4x_1 + 6x_2 + 2x_3 \leq 40 + M(1-y_4) \\
0 \leq x_i \leq My_i, \text{ for } i=1,2,3 \\
x_i \geq 0, \text{ for } i=1, 2, 3  \\
y_i \text{ binary, for } i=1, 2, 3, 4 \\
\end{array}
$

Note that if $y_4 = 0$ we are using Plant 1 and if $y_4 = 1$ we are using Plant 2.

The complete formulation of this problem is discussed on pages 490-491 of the textbook. 

## Pyomo Concrete Formulation Solution

Here we'll just use individual variables $x_1, x_2,$ etc. for maximum transparency.  In the homework you'll need to include abstract formulations, but we include a concrete formulation here to help you understand how binary variables work in Pyomo.  The next section has the abstract formulation.

In [0]:
# concrete Good Products
m = ConcreteModel(name="Example_1")

m.x1 = Var(bounds=(0,7))
m.x2 = Var(bounds=(0,5))
m.x3 = Var(bounds=(0,9))

m.y1 = Var(domain=Boolean)
m.y2 = Var(domain=Boolean)
m.y3 = Var(domain=Boolean)
m.y4 = Var(domain=Boolean) # 0 to use Plant 1, 1 to use Plant 2

m.profit = Objective( expr = 5*m.x1 + 7*m.x2 + 3*m.x3, sense = maximize)

bigM = 10000

# Constraints:
m.cts = ConstraintList()
m.cts.add( m.y1 + m.y2 + m.y3 == 2)
m.cts.add( 3 * m.x1 + 4 * m.x2 + 2 * m.x3 <= 30 + bigM * m.y4 )
m.cts.add( 4 * m.x1 + 6 * m.x2 + 2 * m.x3 <= 40 + bigM * (1 - m.y4))
m.cts.add( m.x1 <= bigM * m.y1)
m.cts.add( m.x2 <= bigM * m.y2)
m.cts.add( m.x3 <= bigM * m.y3)

# Solve
solver = SolverFactory('glpk')
solver.solve(m)

print(f"Maximum Profit = ${1000*m.profit():,.2f}")

print("Use "+ "Plant 2." if m.y4() else "Plant 1." )

for i,amount in enumerate([m.x1(),m.x2(),m.x3()]):
    if amount > 0:
        print(f"Produce {amount:0.1f} of product {i+1} per week")

## Pyomo Abstract Formulation Solution

To make the abstract formulation we add an extra binary variable, $y_5$, so that we have one for each plant to make the plant constraints have the same format.  The math formulation then replaces:

$
\begin{array}{l}
3x_1 + 4x_2 + 2x_3 \leq 30 + My_4 \\
4x_1 + 6x_2 + 2x_3 \leq 40 + M(1-y_4) \\
\end{array}
$

with:

$
\begin{array}{l}
3x_1 + 4x_2 + 2x_3 \leq 30 + M(1-y_4)\\
4x_1 + 6x_2 + 2x_3 \leq 40 + M(1-y_5)\\
y_4 + y_5 = 1
\end{array}
$

so if $y_4 = 1$ we are using Plant 1 and if $y_5 = 1$ we are using Plant 2.

In [0]:
# abstract Good Products

# Problem data
products = ['Product1', 'Product2', 'Product3']
unit_profit = dict(zip(products, [5, 7, 3]))

sales_potential = dict(zip(products, [7, 5, 9]))
def bounds_rule(model, product):
    return ((0, sales_potential[product]))

plants = ['Plant1', 'Plant2']
production_avail = dict(zip(plants, [30, 40]))

tpu = [[3, 4, 2], [4, 6, 2]]
time_per_unit = {
    plants[p]: dict(zip(products, tpu[p][:]))
    for p in range(len(plants))
}
bigM = 10000

num_products_to_choose = 2
num_plants_to_use = 1

# Instantiate concrete model
M = ConcreteModel(name="Example_1")

# Decision Variables
M.x = Var(products, domain=Reals, bounds=bounds_rule)
M.y = Var(products, domain=Boolean)
M.plant_choice = Var(plants, domain=Boolean)

# Objective:  Maximize Profit
M.profit = Objective(expr=sum(unit_profit[pr] * M.x[pr] for pr in products),
                     sense=maximize)

# Constraints:
M.constraints = ConstraintList()

for pr in products:  # produce product only if product is chosen
    M.constraints.add(M.x[pr] <= bigM * M.y[pr])

# choose 2 products
M.constraints.add(sum(M.y[pr] for pr in products) == num_products_to_choose)

for pl in plants:  # production capacities
    M.constraints.add(
        sum(time_per_unit[pl][pr] * M.x[pr]
            for pr in products) <= production_avail[pl] +
        bigM * (1-M.plant_choice[pl]) )

# choose 1 plant
M.constraints.add(sum(M.plant_choice[pl] for pl in plants) == num_plants_to_use)

# Solve
solver = SolverFactory('glpk')
solver.solve(M)

print(f"Maximum Profit = ${1000 * M.profit():,.2f}")

print("\nWhich plant to use:")
for pl in plants:
    print(f"Produce at {pl}? {['No','Yes'][int(M.plant_choice[pl]())]}")

print("\nWhich products and how many:")
for pr in products:
    if bool(M.y[pr]()):
        print(f"Produce {pr} at a rate of {M.x[pr]():.2f} per week")
    else:
        print(f"Do not produce {pr}" )

# Self-Assessments

## *Self Assessment: Solving the California Manufacturing BIP*

You should have read the textbook and watched the first two videos in the storybook before doing this self-assessment.

Use Pyomo in Python to find the solution to the BIP model for the California Manufacturing Company problem in section 12.1 of the Hillier textbook.

**Complete the other videos and the remainder of this notebook before attempting the self-assessments below.**

## *Self-Assessment: Integer Programming*

**True or False:** Integer programs are generally more computationally difficult than linear programs with continuous variables. 

## *Self-Assessment: Type of Programming*

The problem 

Maximize $Z = 7 x_1 + 3 x_2$

Subject to:

$
\begin{array}{ccccc}
 5 x_1 & +  & 7 x_2   & \leq & 27 \\
 4 x_1 & +  &   x_2 & \leq & 14 \\
3x_1 & - & 2x_2 & \leq & 19
\end{array}
$

$x_1 \geq 0$, $x_2 \geq 0$, $x_1$ integer

is an example of a(n)

a. nonlinear program.

b. integer program.

c. mixed integer program.

d. none of the above.


## *Self-Assessment: Rounding Solutions to Integers*

Solving an integer programming problem by rounding off answers obtained by solving it as a linear programming problem, we find that

a. The values of the decision variables obtained by rounding are always very close to the optimal values.

b. The true value of the objective function for a maximization problem will likely be less than that found by solving the linear programming problem.

c. The true value of the objective function for a minimization problem will likely be more than that found by solving the linear programming problem.

d. The lower bound reaches zero.

e. None of the above.


## *Self-Assessment: Either/Or Constraints*

**True or False:** To implement an either/or constraint where one or both of two constraints must be satisfied it is necessary add two binary variables.  

## *Self-Assessment: Number of Solutions in BIP*

**True or False:**  There are $n^2$ solutions to consider when there are $n$ binary decision variables to be considered in an integer programming problem.  